## Closure Applications

In [168]:
class Averager:
    def __init__(self):
        self.numbers = []
        
    def add(self, number):
        self.numbers.append(number)
        total = sum(self.numbers)
        count = len(self.numbers)
        return total / count
#naše jednoduchá classa co průměruje

In [169]:
a = Averager()

In [170]:
a.add(10)

10.0

In [171]:
a.add(20)

15.0

In [172]:
a.add(30)

20.0

Naše classa funguje jak má, vše je ok. Nyní koukneme jak toto vytvořit pomocí closures.

In [185]:
def averager():
    numbers = []
    def add(number):
        numbers.append(number)
        #nonlocal - takže máme closure
        total = sum(numbers)
        count = len(numbers)
        return total / count
    return add

In [186]:
a = averager()

In [187]:
a.__code__.co_freevars

('numbers',)

In [175]:
a(10)

10.0

In [176]:
a(20)

15.0

In [177]:
a(30)

20.0

Takže máme stejný výsledek pomocí closure. To je prostě pěkné jelikož si sdílí svůj list napříč funkcí tak vše frčí jak má.

In [188]:
def averager():
    total = 0
    count = 0
    def add(number):
        nonlocal total
        nonlocal count
        #potřebuji zajistit aby si udrželi svojí hodnotu
        #mám zde assigment proto jim dodávám nonlocal
        total = total + number
        count = count +1
        return total / count
    return add

In [182]:
a = averager()

In [184]:
a.__code__.co_freevars

('count', 'total')

Máme tedy dvě free vars hodnoty.

In [189]:
class Averager:
    def __init__(self):
        self.total = 0
        self.count = 0
        
    def add(self, number):
        self.total += number
        self.count += 1
        return self.total / self.count
#náš averager pomocí classy podruhé

In [190]:
from time import perf_counter

In [191]:
perf_counter()

79738.3593392

In [192]:
perf_counter()

79748.3825066

Jen náč časovač.

In [193]:
class Timer:
    def __init__(self):
        self.start = perf_counter()
        #v podstatě měřič naší inicializace
        
    def poll(self):
        return perf_counter() - self.start
    #pouze odečet inicializace classy a zavolání metody poll

In [194]:
t1 = Timer() #a ihned vytvoří perf_counter start

In [196]:
t1.poll() #cca 6 sekund rozdíl

12.569548100000247

In [198]:
t1.poll() #a nyní 18 sekund :)

26.439224200003082

Nyní se metodu poll změním na __call__ - jen když zavolám classu

In [205]:
class Timer:
    def __init__(self):
        self.start = perf_counter()
        #v podstatě měřič naší inicializace
        
    def __call__(self):
        return perf_counter() - self.start
    #pouze odečet inicializace classy a zavolání metody poll

In [206]:
t1 = Timer()

In [208]:
t1() #teď stačí jen volat classu a spustím naší funkci

6.01990789998672

Nyní si vytvoříme closure co udělá to samé

In [209]:
def timer():
    start = perf_counter()
    def poll():
        return perf_counter() - start
    return poll

In [211]:
t2 = timer()

In [212]:
t2()

2.1699661999882665

A máme to samé :). Pokračujeme v dalších aplikacích.

In [228]:
def counter(initial_value=0):
    def inc(increment=1):
        nonlocal initial_value
        initial_value += increment
        return initial_value
    return inc
#poměrně jasná funkce
#jelikož mám assigment musím zajistit že máme nonlocal

In [223]:
counter1 = counter()

In [225]:
counter1()

2

In [252]:
def counter(fn):
    cnt = 0
    """funkce pro počítání spuštění funkce"""
    def inner(*args, **kwargs):
        nonlocal cnt
        cnt += 1 #assigment musím dát nonlocal
        print(f"{fn.__name__} that has been called {cnt}")
        return fn(*args, **kwargs)
    return inner
    #v podstatě funkci spišutíme přes counter a díky tomu jí můžeme počítat :)

In [253]:
def add(a,b):
    return a + b

In [254]:
def mult(a,b):
    return a*b

In [255]:
counter_add = counter(add) #stačí mů jen název funkce

In [256]:
counter_add.__closure__

(<cell at 0x000001482B94C250: int object at 0x00007FF85C0026F0>,
 <cell at 0x000001482B94C790: function object at 0x000001482B959280>)

In [257]:
counter_add.__code__.co_freevars

('cnt', 'fn')

Jasné naše funkce je closure s počtem a funkcí. Funkce nemá assigment proto neřeším nonlocal.

In [258]:
counter_add(10,20)

add that has been called 1


30

In [259]:
counter_add(20,30)

add that has been called 2


50

Zde je vlastně důležité, že naše funkce se spouští skrze counter - jinou funkci a ta přijímá *args a **kwargs právě proto aby mohla přijmout jakýkoliv druh funkce - s jakýmikoliv parametry. A jako bonus navíc nám to počítá četnost spuštění.

Vytvoříme si global dictionary, který nám bude naší evidenci udržovat - očetnosti funkcí.

In [260]:
counters = dict()

In [267]:
def counter(fn):
    cnt = 0
    def inner(*args,**kwargs):
        nonlocal cnt
        cnt += 1
        counters[fn.__name__] = cnt
        #vkládáme do dictionary
        #counters si poradí sám 
        return fn(*args, **kwargs)
    return inner
#takto mů§j counter ukládá všechno do dicitonary který je ale global
#takžed můj dictionary trackuje všechny funkce které budou přes něj spuštěny

In [269]:
counted_add = counter(add)
counter_mult = counter(mult)

In [270]:
counted_add(10,20)

30

In [271]:
counted_add(20,40)

60

In [272]:
counters

{'add': 2}

In [274]:
counter_mult(2,5)

10

In [275]:
counters

{'add': 2, 'mult': 1}

Takže to už je docela pěkné.

Ideální by bylo asi vytvářet dictionary ve funkci? Nemusím si tak pamatovat a nic definovat v global.

In [276]:
def counter(fn, counters):
    cnt = 0
    def inner(*args,**kwargs):
        nonlocal cnt
        cnt += 1
        counters[fn.__name__] = cnt
        return fn(*args, **kwargs)
    return inner
#nyní máme counters v parametrech takže přebijeme global
#a znovu neřeším u counters nonlocal protože nemodifikujeme counters
#my modifikujeme jeho obsah

In [277]:
c = dict()

In [278]:
counted_add = counter(add, c)
counted_mult = counter(mult,c)

In [280]:
counted_add(20,10)

30

In [281]:
counted_add(10,15)

25

In [282]:
counted_mult(2,5)

10

In [283]:
counted_mult(3,6)

18

In [285]:
counters #stále stejné ten jsme neměnili

{'add': 2, 'mult': 1}

In [287]:
c #tohle je náš nový

{'add': 2, 'mult': 2}

Na závěr si dáme faktorial

In [288]:
def fact(n):
    product = 1
    for i in range(2, n+1):
        product *= i
    return product

In [289]:
fact(3)

6